### nn.Module.register_buffer and nn.Parameters
nn.Module.register_buffer特性：
1. 不会产生梯度require_grad = False
2. 不会注册到模型参数中model.parameters()
3. 会注册到模型model.state_dict()中。

In [1]:
import torch
import torchvision

In [ ]:
# 将需要保存到state_dict的变量定义为register_buffer，比如说量化参数。（见quantization.ipynb:QParam）
self.register_buffer('scale', torch.tensor([], requires_grad=False))  
self.register_buffer('zero_point', torch.tensor([], requires_grad=False))
self.register_buffer('min', torch.tensor([], requires_grad=False))
self.register_buffer('max', torch.tensor([], requires_grad=False))

# 对于需要在设备之间转移的变量，需要时nn.Module的子类，所以必须将其注册为nn.Parameter 或者 buffer，比如说self.M(见quantization.ipynb:QConv2d:freeze)
self.register_buffer('M', torch.tensor([], requires_grad=False))
self.M.data = (self.qw.scale*self.qi.scale / self.qo.scale).data  #赋值tensor而不是对象本身


### device

在设备之间转移模型和数据

In [14]:
data = torch.zeros(1,2)
model = torchvision.models.resnet18()

# 模型所在的设备，需要查看其参数所在的设备
print("data's device:{},model's device:{}".format(data.device,next(model.parameters()).device))

# 将数据进行转移，必须使用赋值
data = data.cuda()
data = data.to("cuda")

# data.cuda()  #error! tensor只使用这句是不够的
print("data's device:",data.device)

# 将model转移到cuda,不需要赋值，赋值也可以
model.cuda()
model.to("cuda")
model = model.cuda()
model = model.to("cuda")
print("model's device:",next(model.parameters()).device)

data's device:cpu,model's device:cpu
data's device: cuda:0
model's device: cuda:0


指定使用的显卡
在shell 环境中
```shell
CUDA_VISIBLE_DEVICES="1,2" python train.py
```

In [ ]:
# 在python代码中
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "1,2"


### Pytorch->onnx

In [ ]:
import torch.onnx 

#Function to Convert to ONNX 
def Convert_ONNX(): 

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  
    input_size = (3,224,224)
    dummy_input = torch.randn(1, *input_size, requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "resnet18.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['input'],   # the model's input names 
         output_names = ['output'], # the model's output names 
         dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes 
                                'output' : {0 : 'batch_size'}}) 
    print(" ") 
    print('Model has been converted to ONNX') 
Convert_ONNX()

使用onnxsim对onnx模型进行简化，安装方式：
```shell
pip install onnxsim
```
有两种执行方式：
1. 在终端执行
```shell
onnxsim old.onnx sim.onnx
```
2. 在python代码中


In [ ]:
onnx_model = onnx.load(f'{os.path.join(cfg.FP32_BASE_PATH, name)}-FP32.onnx') 
model_simp, check = simplify(onnx_model)   #对onnx模型进行简化，消除冗余算子        
assert check, "Simplified ONNX model could not be validated"
onnx.save(model_simp, f'{os.path.join(cfg.FP32_BASE_PATH, name)}-FP32.onnx')